In [2]:
import pandas as pd
import numpy as np

# Set seed for reproducibility
np.random.seed(42)

# Generate 180 days of data (6 months)
dates = pd.date_range(start='2024-01-01', periods=180, freq='D')
n = len(dates)

# Simulating Operational Data
# Flow Rate (BFPD) - slight fluctuation
flow_rate = np.random.normal(loc=1000, scale=50, size=n)

# Frequency (Hz) - Operator occasionally adjusts speed
freq = np.random.choice([55, 58, 60], size=n, p=[0.2, 0.3, 0.5])

# Specific Gravity (SG) of fluid
sg = 0.85

# Manufacturer Curve Parameters (Base 60Hz)
# Head = a*Q^2 + b*Q + c
a_curve, b_curve, c_curve = -0.0005, 0.1, 2500

# Calculate THEORETICAL Head at current conditions using Affinity Laws
# Q_60 = Q_act * (60 / f)
# H_60 = H_act * (60 / f)^2
flow_60 = flow_rate * (60 / freq)
theoretical_head_60 = (a_curve * flow_60**2) + (b_curve * flow_60) + c_curve
theoretical_head_actual = theoretical_head_60 * (freq / 60)**2

# Simulate DEGRADATION (ACTUAL HEAD)
# Healthy for first 60 days, then linear degradation to 85% health
health_factor = np.ones(n)
health_factor[60:] = np.linspace(1.0, 0.85, n-60)

# Add noise to actual head reading
actual_head = theoretical_head_actual * health_factor + np.random.normal(0, 20, n)

# Calculate Intake and Discharge Pressure (Reverse engineering for the CSV)
# TDH = (P_discharge - P_intake) * 2.31 / SG
# Let's assume constant Intake Pressure (Pi) for simplicity
p_intake = np.random.normal(500, 10, n)
p_discharge = p_intake + (actual_head * sg / 2.31)

# Create DataFrame
df = pd.DataFrame({
    'Date': dates,
    'Frequency_Hz': freq,
    'Flow_Rate_BFPD': flow_rate.astype(int),
    'Intake_Pressure_Psi': p_intake.round(2),
    'Discharge_Pressure_Psi': p_discharge.round(2),
    'Fluid_SG': sg
})

# Save to CSV
df.to_csv('pump_operations_data.csv', index=False)
print("CSV Generated Successfully.")

CSV Generated Successfully.
